In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
#from tqdm import tqdm
import eli5


In [3]:
from sklearn.ensemble import RandomForestRegressor

Loading data



In [4]:
train_bike = pd.read_csv("train.csv")

#test_bike = pd.read_csv("test.csv")

In [5]:
train_bike["datetime"].head()

0    2011-01-01 00:00:00
1    2011-01-01 01:00:00
2    2011-01-01 02:00:00
3    2011-01-01 03:00:00
4    2011-01-01 04:00:00
Name: datetime, dtype: object

In [6]:
train_bike["datetime"] = pd.to_datetime(train_bike["datetime"])

train_bike["datetime"].describe()

count                   10886
unique                  10886
top       2011-06-09 04:00:00
freq                        1
first     2011-01-01 00:00:00
last      2012-12-19 23:00:00
Name: datetime, dtype: object

In [7]:
def transform_dates(dataframe):
    dataframe["datetime"] = pd.to_datetime(dataframe["datetime"])
    dataframe["dayofweek"] = dataframe["datetime"].dt.dayofweek      #The day of the week with Monday=0, Sunday=6
    dataframe["year"] = dataframe["datetime"].dt.year
    dataframe["month"] = dataframe["datetime"].dt.month
    dataframe["day"] = dataframe["datetime"].dt.day
    dataframe["hour"] = dataframe["datetime"].dt.hour
    return dataframe

In [8]:
train_bike = transform_dates(train_bike)

In [9]:
X = train_bike.drop(["count", "casual", "registered", "datetime"], axis = 1)
y = train_bike["registered"]

In [10]:
from sklearn.model_selection import train_test_split #dzielnie na zetsawy
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, random_state=42)

In [11]:
# best_score = 0
# parameters = {}

In [12]:
#from tqdm import tqdm

In [13]:
# for n_estimators in tqdm(range(2)):
#     for max_features in np.arange(0.2, 1.01, 0.2):
#         for min_samples_split in range(2,21):
#             for min_samples_leaf in range(1,21):
#                 for criterion in ["mse", "mae"]:
#                     for bootstrap in [True, False]:
#                         clf_rfr = RandomForestRegressor(n_estimators = n_estimators, max_features = max_features, min_samples_split = min_samples_split, 
#                                                      min_samples_leaf = min_samples_leaf, criterion = criterion,
#                                                       bootstrap = bootstrap )
#                         print("jhkjhkj")
#                         try:
#                             scores = cross_val_score(clf_rfr, X_train, y_train, cv = 5)
#                         except ValueError as e:
#                             print(e)
#                             continue
#                         if scores.mean() > best_score:
#                             #print("New best parameters!")
#                             parameters["n_estimators"] = n_estimators
#                             parameters["best_features"] = max_features
#                             parameters["best_min_split"] = min_samples_split
#                             parameters["best_min_leaf"] = min_samples_leaf
#                             parameters["best_criterion"] = criterion
#                             parameters["bootstrap"] = bootstrap
#                             best_score = scores.mean()
#                             print("Accuracy: {}".format(scores.mean()))
#                             print("n_estimators: {}, max_features: {}, min_samples_split: {}, min_samples_leaf: {}, criterion: {}, bootstrap: {}".format(n_estimators, max_features, 
#                                                                                                                      min_samples_split, 
#                                                                                                              min_samples_leaf, 
#                                                                                                              criterion, bootstrap))

In [14]:
parameters = {'n_estimators': [20], 'max_features': np.arange(0.1, 1.01, 0.1),
              'bootstrap': [True, False], 'criterion': ["mse", "mae"]}
forest = RandomForestRegressor()

In [15]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y))**2))

rmsle_score = make_scorer(rmsle, greater_is_better = False)

In [16]:
regressor = GridSearchCV(estimator = forest, param_grid = parameters, 
                         return_train_score=True, scoring = rmsle_score)

In [17]:
#clf_rfr = RandomForestRegressor(criterion = 'mse')

In [18]:
regressor.fit(X = X_train, y = y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20], 'max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]), 'bootstrap': [True, False], 'criterion': ['mse', 'mae']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(rmsle, greater_is_better=False), verbose=0)

In [19]:
eli5.explain_weights(regressor.best_estimator_, feature_names=X_train.columns.values)

Weight,Feature
0.5900 ± 0.0120,hour
0.1275 ± 0.0342,workingday
0.0910 ± 0.0108,year
0.0603 ± 0.0130,month
0.0264 ± 0.0344,dayofweek
0.0248 ± 0.0178,temp
0.0232 ± 0.0092,humidity
0.0207 ± 0.0167,atemp
0.0142 ± 0.0081,weather
0.0081 ± 0.0030,day


In [20]:
predicted = regressor.predict(X_test) #robimy predyckję
wyniki = pd.DataFrame(y_test) 
wyniki["y_pred"] = predicted #dokljamy kolumnę z predykcją
wyniki.rename(columns = {"Klasa":"y_test"}, inplace = True)

In [21]:
wyniki

,registered,y_pred
3133,98,95.15
5786,13,38.30
5224,151,139.05
8953,163,152.75
8054,176,141.20
10044,135,152.35
5337,135,120.50
2753,182,179.50
10127,583,645.20
33,46,53.50


In [22]:
wewnetrzny_test = rmsle(y_pred = wyniki["y_pred"], y = wyniki["registered"])
print(wewnetrzny_test)

0.3208863937411223


In [28]:
kaggle = pd.read_csv("test.csv")
to_submit = kaggle

In [29]:
kaggle = transform_dates(kaggle)
kaggle = kaggle.drop(["datetime"], axis = 1)

In [32]:
to_submit["registered"] = regressor.predict(kaggle)

In [33]:
to_submit

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,dayofweek,year,month,day,hour,count,registered
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,3,2011,1,20,0,13.55,13.55
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,3,2011,1,20,1,5.50,5.50
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,3,2011,1,20,2,6.05,6.05
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,3,2011,1,20,3,2.70,2.70
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,3,2011,1,20,4,2.50,2.50
5,2011-01-20 05:00:00,1,0,1,1,9.84,11.365,60,15.0013,3,2011,1,20,5,4.20,4.20
6,2011-01-20 06:00:00,1,0,1,1,9.02,10.605,60,15.0013,3,2011,1,20,6,33.00,33.00
7,2011-01-20 07:00:00,1,0,1,1,9.02,10.605,55,15.0013,3,2011,1,20,7,90.90,90.90
8,2011-01-20 08:00:00,1,0,1,1,9.02,10.605,55,19.0012,3,2011,1,20,8,213.45,213.45
9,2011-01-20 09:00:00,1,0,1,2,9.84,11.365,52,15.0013,3,2011,1,20,9,114.75,114.75


In [ ]:
#prediction_rfr = clf_rfr.predict(X_test)

In [ ]:
#prediction_rfr